<a href="https://colab.research.google.com/github/physicistharish/GAN_Event_Generator/blob/main/GAN_event_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import os
from sklearn.preprocessing import MinMaxScaler
import glob
import imageio
import matplotlib.pyplot as plt
import PIL
from tensorflow.keras import layers
import time
import io
import pickle
from IPython import display
from google.colab import files

In [ ]:
from keras import backend

from numpy import cov
from numpy import trace
from numpy import iscomplexobj
from numpy.random import random
from scipy.linalg import sqrtm

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

path = "/content/drive/MyDrive/Project_Files/Csv/16349 events XY.csv"# This line is not required if the input dataset has the proper 13 columns

df1 = pd.read_csv(path)
print(df1)


       Unnamed: 0         0         1  ...        10        11        12
0               0  0.388266  1.066190  ...  -12.1232  -36.5231  -76.7150
1               0 -0.450752 -0.833231  ...   24.5506   32.9202   40.7282
2               0  1.080110  2.199080  ...   42.8811   61.5420   89.7588
3               0 -2.602280 -7.072710  ...  -40.4210  -47.4739  -66.2680
4               0  0.085928  0.266413  ...  108.9250  105.9890   95.7408
...           ...       ...       ...  ...       ...       ...       ...
16344           0  0.424090  0.911776  ...   17.8851   16.7183   18.3112
16345           0  0.210162  2.838150  ...   20.9781   24.7027   34.6982
16346           0  0.570230  1.158780  ...   69.3984   86.1083  108.1730
16347           0  0.645774  2.287450  ...   17.2922   14.9753  -19.0565
16348           0  0.895050  3.375180  ...   19.4355   31.8020   33.9234

[16349 rows x 14 columns]


In [ ]:
df1 = df1.drop(columns='Unnamed: 0') # This line is not required if the input dataset has the proper 13 columns
print(df1)

              0         1         2  ...        10        11        12
0      0.388266  1.066190   1.65756  ...  -12.1232  -36.5231  -76.7150
1     -0.450752 -0.833231  -1.64948  ...   24.5506   32.9202   40.7282
2      1.080110  2.199080   4.15661  ...   42.8811   61.5420   89.7588
3     -2.602280 -7.072710 -10.71260  ...  -40.4210  -47.4739  -66.2680
4      0.085928  0.266413   2.94087  ...  108.9250  105.9890   95.7408
...         ...       ...       ...  ...       ...       ...       ...
16344  0.424090  0.911776   1.41807  ...   17.8851   16.7183   18.3112
16345  0.210162  2.838150   7.73274  ...   20.9781   24.7027   34.6982
16346  0.570230  1.158780   1.30524  ...   69.3984   86.1083  108.1730
16347  0.645774  2.287450   2.81709  ...   17.2922   14.9753  -19.0565
16348  0.895050  3.375180   3.97640  ...   19.4355   31.8020   33.9234

[16349 rows x 13 columns]


In [ ]:
def z_score(df):
    # copy the dataframe
    df_std = df.copy()
    df_std = df_std.transpose()
    # apply the z-score method
    for column in df_std.columns:
        df_std[column] = (df_std[column] - df_std[column].mean()) / df_std[column].std()
    df_std = df_std.transpose()   
    return df_std

In [ ]:
df1=z_score(df1)

In [ ]:
BUFFER_SIZE = 16300
BATCH_SIZE = 1

train_dataset_new = tf.data.Dataset.from_tensor_slices(df1).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

In [ ]:
def make_generator_model():
    model = tf.keras.Sequential()
    
    model.add(layers.Dense(4, use_bias=False, input_shape=(100,)))
    model.add(layers.LeakyReLU())

    model.add(layers.Dense(2*64,use_bias=False))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.2))

    model.add(layers.Dense(2*64,use_bias=False))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.2))
    
    model.add(layers.Dense(13,use_bias=False))
    model.add(layers.LeakyReLU())

    model.add(layers.Reshape((13,1)))
    
    return model

In [ ]:
generator = make_generator_model()
# Trying out a sample generation
noise = tf.random.normal([1, 100])
generated_image = generator(noise, training=False)

In [ ]:
def make_discriminator_model():
    model = tf.keras.Sequential()

    model.add(layers.Flatten())
    model.add(layers.Dense(13,use_bias=False))
    model.add(layers.LeakyReLU())
    
    model.add(layers.Dense(64,use_bias=False))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.2))
      
    model.add(layers.Dense(64,use_bias=False))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.2))
   
    model.add(layers.Dense(1,use_bias=False))
    model.add(layers.LeakyReLU())

    return model

In [ ]:
discriminator = make_discriminator_model()
decision = discriminator(generated_image)
print (decision)

In [ ]:
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

In [ ]:
def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

In [ ]:
def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

In [ ]:
generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

In [ ]:
EPOCHS = 15
noise_dim = 100
num_examples_to_generate = 10

seed = tf.random.normal([num_examples_to_generate, noise_dim])

In [ ]:
@tf.function
def train_step(images,n):
    noise = tf.random.normal([BATCH_SIZE, noise_dim])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
      generated_images = generator(noise, training=True)

      real_output = discriminator(images, training=True)
      fake_output = discriminator(generated_images, training=True)
      
      gen_loss = generator_loss(fake_output)
      disc_loss = discriminator_loss(real_output, fake_output)
      
    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

In [ ]:
def train(dataset, epochs):
  for i in range(epochs):
    start = time.time()

    for image_batch in dataset:
      train_step(image_batch,i)

    pddf = generate_and_save_images(generator, i + 1,seed)
    print('Time for epoch {} is {} sec'.format(i + 1, time.time()-start))
    if i == epochs-1:
      pddf.to_csv('file_name.csv'.format(i+1),mode = 'a')
      files.download('file_name.csv'.format(i+1))

In [ ]:
def generate_and_save_images(model, test_input):
  predictions = model(test_input, training=False)
   
  predictions = tf.reshape(predictions,[num_examples_to_generate,13])
  predict_data = predictions.numpy()
  panda_df = pd.DataFrame(data = predict_data)

  return panda_df

In [ ]:
train(train_dataset_new, EPOCHS)

In [ ]:
new_df = generate_and_save_images(generator,tf.random.normal([20000,100]))

In [ ]:
new_df.to_csv('20000 generated posx.csv')